In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

# Иницијализација на Web Driver за Selenium
driver = webdriver.Chrome()
driver.get("https://kosuli.mk")
wait = WebDriverWait(driver, 10)

# Дефинирање на речник, dictionary на категориите кои ги скрејпам со големи букви
menu_targets = {
    "Брендови": {"CASA MODA", "FF-FASHION", "VENTI"},
    "Кратки ракави": {"МОДЕРН ФИТ"},
}

# Празно dictionary за чување на УРЛ's
extracted_urls = {}

for top_menu, targets in menu_targets.items():
    # Се чека елементот да биде пронајден со hover 
    link = wait.until(EC.presence_of_element_located(
        (By.XPATH, f"//a[contains(normalize-space(), '{top_menu}')]")
    ))
    actions = ActionChains(driver)
    actions.move_to_element(link).perform()

    # Се чеза за елементот да биде видлив за да може да се лоцира
    submenu = wait.until(EC.presence_of_element_located(
        (By.XPATH, f"//li[a[contains(normalize-space(), '{top_menu}')]]/ul")
    ))

    # Се зачувуваат сите линкови од секоја категорија 
    submenu_anchors = submenu.find_elements(By.TAG_NAME, "a")

    for a in submenu_anchors:
        text = a.text.strip().upper()
        href = a.get_attribute("href")
        if text in targets and href:
            # Се зачувува во варијабла линкот 
            key = text.lower().replace(" ", "_")
            extracted_urls[key] = href

# Затварање на бровсерот односно исклучување на селениум
driver.quit()

# Зачувување на варијабли, главната варијбала страницата и потоа секое урл посебно
base_url = "https://kosuli.mk"
url_casa_moda = extracted_urls.get("casa_moda")
url_ff_fashion = extracted_urls.get("ff-fashion")
url_venti = extracted_urls.get("venti")
url_modern_fit = extracted_urls.get("модерн_фит")

# Принтање на секој линк
print("Casa Moda :", url_casa_moda)
print("FF-Fashion:", url_ff_fashion)
print("Venti     :", url_venti)
print("Modern Fit:", url_modern_fit)


Casa Moda : https://kosuli.mk/product-category/casa-moda/
FF-Fashion: https://kosuli.mk/product-category/f-fashion/
Venti     : https://kosuli.mk/product-category/venti/
Modern Fit: https://kosuli.mk/product-category/modern-fit/


In [4]:
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Иницијализација на селениум веб драјверот
options = webdriver.ChromeOptions()
prefs = {"profile.managed_default_content_settings.images": 2}
options.add_experimental_option("prefs", prefs)

driver = webdriver.Chrome(options=options)
driver.implicitly_wait(10)
wait = WebDriverWait(driver, 10)

# Скрејпање на сите урл на продукти и нивно итерирање
product_urls = []
page = 1

while True:
    driver.get(f"{url_casa_moda}page/{page}/")
    try:
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "ul.products li.product")))
    except:
        break

    links = driver.find_elements(By.CSS_SELECTOR, "ul.products li.product a.woocommerce-LoopProduct-link")
    if not links:
        break

    for a in links:
        href = a.get_attribute("href")
        if href and href not in product_urls:
            product_urls.append(href)

    try:
        driver.find_element(By.CSS_SELECTOR, "a.next.page-numbers")
        page += 1
    except NoSuchElementException:
        break

print(f"→ Found {len(product_urls)} products across {page} pages.")

# Скрејпање на секој продукт негово итерирање
data = []
for idx, href in enumerate(product_urls, start=1):
    driver.get(href)
    try:
        wait.until(EC.presence_of_element_located((By.TAG_NAME, "h1")))
    except:
        print(f"Skipping {href} (no h1 tag found)")
        continue

    # Скрејпање на името
    name = driver.find_element(By.TAG_NAME, "h1").text.strip()

    # Скрејпање на цената
    try:
        price = driver.find_element(By.CSS_SELECTOR, "p.price").text.strip()
    except NoSuchElementException:
        price = ""

    # Скрејпање на урлто на слиакта на секој продукт
    try:
        img = driver.find_element(By.CSS_SELECTOR, "div.woocommerce-product-gallery__image img")
        image_url = img.get_attribute("data-src") or img.get_attribute("src")
    except NoSuchElementException:
        image_url = ""

    # Скрејпање на големините со што се скрола надоле за да се пронајдат величините каде се достапни
    sizes = []
    try:
        descr = driver.find_element(By.ID, "tab-description")
        driver.execute_script("arguments[0].scrollIntoView(true);", descr)
        div = descr.find_element(By.XPATH, ".//div[contains(text(),'Достапни големини')]")
        ul = div.find_element(By.XPATH, "following-sibling::ul[1]")
        sizes = [li.text.strip() for li in ul.find_elements(By.TAG_NAME, "li") if li.text.strip()]
    except Exception:
        try:
            descr = driver.find_element(By.ID, "tab-description")
            driver.execute_script("arguments[0].scrollIntoView(true);", descr)
            ul = descr.find_element(By.TAG_NAME, "ul")
            sizes = [li.text.strip() for li in ul.find_elements(By.TAG_NAME, "li") if li.text.strip()]
        except Exception:
            sizes = []

    sizes_str = ", ".join(sizes)

    print(f"[{idx}/{len(product_urls)}] {name} → Sizes: {sizes_str or '—'}")

    data.append({
        "Name": name,
        "Price": price,
        "Sizes": sizes_str,
        "Image_URL": image_url,
        "Product_URL": href
    })

# Зачувување во Пандас датафрејм во цсв формат и нивно принтање секој продукт посебно со неговото име и величини
df = pd.DataFrame(data)
df.to_csv("kosuli_casa_moda.csv", index=False, encoding="utf-8")

print(f"✔ Scraped {len(df)} products. Saved to kosuli_casa_moda.csv.")
driver.quit()


→ Found 37 products across 4 pages.
[1/37] CM098 REGULAR FIT LONG SLEEVE BLUE → Sizes: L, XXL, 4XL
[2/37] CM095 REGULAR FIT LONG SLEEVE SILVER → Sizes: XXL
[3/37] CM094 REGULAR FIT LONG SLEEVE RED/SILVER → Sizes: XXL,
[4/37] CM091 REGULAR FIT LONG SLEEVE BLUE/WHITE → Sizes: L,  XXL,
[5/37] CM089 REGULAR FIT LONG SLEEVE ORANGE → Sizes: XL, XXL,  4XL
[6/37] CM088 REGULAR FIT LONG SLEEVE RED → Sizes: XL, XXL
[7/37] CM087 REGULAR FIT LONG SLEEVE → Sizes: XXL
[8/37] CM086 REGULAR FIT LONG SLEEVE BLUE → Sizes: 2XL,
[9/37] CM083 REGULAR FIT LONG SLEEVE BLUE → Sizes: L, XL, XXL, 4XL, 5XL
[10/37] CM082 REGULAR FIT LONG SLEEVE RED → Sizes: XXL, 5XL
[11/37] CM081 REGULAR FIT LONG SLEEVE BLUE → Sizes: XXL,
[12/37] CM065 REGULAR FIT LONG SLEEVE BLUE → Sizes: 5XL
[13/37] CM055 REGULAR FIT LONG SLEEVE RED → Sizes: L, XL
[14/37] CM054 REGULAR FIT LONG SLEEVE PINK\LEN → Sizes: L
[15/37] СМ052 REGULAR FIT LONG SLEEVE PINK OXFORD → Sizes: L, XL
[16/37] CM050 REGULAR FIT LONG SLEEVE BLUE/LEN PRINT → Sizes